## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import and read the charity_data csv
application_df = pd.read_csv("/content/drive/MyDrive/Colab/Homework/Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"], axis=1)

In [5]:
# Determine the number of unique values in each column.
application_df.nunique(axis=0)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Determine the number of unique values in each column.
for col in application_df.columns:
  if col not in ["STATUS", "ASK_AMT"]:
    print(col)
    print(len(application_df[col].value_counts()))
    print()

APPLICATION_TYPE
17

AFFILIATION
6

CLASSIFICATION
71

USE_CASE
5

ORGANIZATION
4

INCOME_AMT
9

SPECIAL_CONSIDERATIONS
2

IS_SUCCESSFUL
2



In [7]:
# Look at APPLICATION_TYPE value counts for binning
application_df.APPLICATION_TYPE.value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
app_vals = application_df.APPLICATION_TYPE.value_counts()

cutoff = 100
application_types_to_replace = list(app_vals[app_vals < cutoff].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2561        1
C1580        1
C2190        1
C1728        1
C1283        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df.CLASSIFICATION.value_counts().loc[lambda x : x>1] 

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1278       10
C1238       10
C1237        9
C1235        9
C7210        7
C1720        6
C2400        6
C4100        6
C1600        5
C1257        5
C2710        3
C0           3
C1260        3
C3200        2
C1234        2
C1256        2
C1246        2
C1267        2
Name: CLASSIFICATION, dtype: int64

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
class_vals = application_df.CLASSIFICATION.value_counts()

cutoff = 150
classifications_to_replace = list(class_vals[class_vals < cutoff].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1003
C7000      777
C1700      287
C4000      194
Name: CLASSIFICATION, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [13]:
application_df["IS_SUCCESSFUL"].value_counts()

1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64

In [16]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['STATUS', 'IS_SUCCESSFUL'], axis=1) # dropping 'STATUS' column for first step in optimization

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [19]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=2, activation="relu", input_dim=46))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=3, activation="tanh"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 94        
                                                                 
 dense_1 (Dense)             (None, 3)                 9         
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 107
Trainable params: 107
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6366 - accuracy: 0.6593
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5861 - accuracy: 0.7189
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5817 - accuracy: 0.7202
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5788 - accuracy: 0.7228
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5769 - accuracy: 0.7221
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5750 - accuracy: 0.7208
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5742 - accuracy: 0.7213
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5732 - accuracy: 0.7214
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5716 - accuracy: 0.7212
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5704 - accura

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5627 - accuracy: 0.7291 - 410ms/epoch - 2ms/step
Loss: 0.5626797080039978, Accuracy: 0.7290962338447571


Slightly improved accuracy and slightly improved loss after removing 'STATUS' column

In [25]:
# Export our model to HDF5 file
nn_model.save('/content/drive/MyDrive/Colab/Homework/AlphabetSoupCharity_Optimization_01.h5')

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=15, activation="relu", input_dim=46)) # changing 'units' parameter

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=10, activation="tanh")) # changing 'units' parameter

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 15)                705       
                                                                 
 dense_4 (Dense)             (None, 10)                160       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 876
Trainable params: 876
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [28]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5847 - accuracy: 0.7035
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5593 - accuracy: 0.7273
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5543 - accuracy: 0.7288
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5518 - accuracy: 0.7300
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5508 - accuracy: 0.7304
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5496 - accuracy: 0.7309
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5487 - accuracy: 0.7311
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5481 - accuracy: 0.7313
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5475 - accuracy: 0.7311
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5475 - accura

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5533 - accuracy: 0.7294 - 390ms/epoch - 1ms/step
Loss: 0.5532656311988831, Accuracy: 0.7294460535049438


Slightly improved accuracy and slightly improved loss after changing 'units' parameter

In [30]:
# Export our model to HDF5 file
nn_model.save('/content/drive/MyDrive/Colab/Homework/AlphabetSoupCharity_Optimization_02.h5')

In [31]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=15, activation="relu", input_dim=46))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=10, activation="tanh"))

# adding additional layer
nn_model.add(tf.keras.layers.Dense(units=10, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 15)                705       
                                                                 
 dense_7 (Dense)             (None, 10)                160       
                                                                 
 dense_8 (Dense)             (None, 10)                110       
                                                                 
 dense_9 (Dense)             (None, 1)                 11        
                                                                 
Total params: 986
Trainable params: 986
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [33]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5748 - accuracy: 0.7147
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5571 - accuracy: 0.7270
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5538 - accuracy: 0.7283
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5524 - accuracy: 0.7281
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5513 - accuracy: 0.7297
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5500 - accuracy: 0.7285
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7293
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5488 - accuracy: 0.7298
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accuracy: 0.7292
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accura

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5531 - accuracy: 0.7291 - 398ms/epoch - 1ms/step
Loss: 0.5531316995620728, Accuracy: 0.7290962338447571


Slightly reduced accuracy and slightly improved loss after adding another layer

In [35]:
# Export our model to HDF5 file
nn_model.save('/content/drive/MyDrive/Colab/Homework/AlphabetSoupCharity_Optimization_03.h5')